# TSA Chapter 7: Interest Rate Cointegration

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch7/TSA_ch7_interest_rates_coint/TSA_ch7_interest_rates_coint.ipynb)

US 10-Year Treasury (GS10) and 3-Month T-Bill (TB3MS) rates from FRED as a cointegrated pair.

In [ ]:
!pip install numpy pandas matplotlib statsmodels pandas_datareader yfinance -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller, coint
import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']
plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})
def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
# Fetch real data from FRED
start, end = '2000-01-01', '2024-12-31'
gs10 = web.DataReader('GS10', 'fred', start, end).dropna()
tb3ms = web.DataReader('TB3MS', 'fred', start, end).dropna()

# Align on common dates (monthly)
data = pd.DataFrame({'GS10': gs10['GS10'], 'TB3MS': tb3ms['TB3MS']}).dropna()
print(f'Period: {data.index[0].strftime("%Y-%m")} to {data.index[-1].strftime("%Y-%m")}, n={len(data)}')

# Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Panel 1: Both rates
axes[0].plot(data.index, data['GS10'], color=BLUE, lw=1, label='10-Year Treasury (GS10)')
axes[0].plot(data.index, data['TB3MS'], color=RED, lw=1, label='3-Month T-Bill (TB3MS)')
axes[0].set_title('US Interest Rates (FRED)', fontweight='bold')
axes[0].set_ylabel('Rate (%)')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)

# Panel 2: Term spread
spread = data['GS10'] - data['TB3MS']
axes[1].plot(data.index, spread, color=GREEN, lw=1)
axes[1].axhline(spread.mean(), color=GRAY, ls='--', lw=0.8, label=f'Mean = {spread.mean():.2f}%')
axes[1].fill_between(data.index, 0, spread, where=spread < 0, color=RED, alpha=0.15, label='Inverted')
axes[1].set_title('Term Spread: GS10 - TB3MS', fontweight='bold')
axes[1].set_ylabel('Spread (%)')
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)

plt.tight_layout()
save_chart(fig, 'interest_rates_coint')
plt.show()

# Cointegration test
t_stat, p_val, _ = coint(data['GS10'], data['TB3MS'])
print(f'\nEngle-Granger cointegration test: t-stat={t_stat:.3f}, p-value={p_val:.4f}')